In [1]:
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision
import numpy as np

D:\CS\Python\anaconda\lib\site-packages\torchvision\_C.pyd
not here
D:\CS\Python\anaconda\lib\site-packages\torchvision\image.pyd
not here


In [10]:
# Convolution Block
def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3, 
                     stride=stride, padding=1, bias=False)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample
        
    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out

# Main Block ResNet
class ResNet(nn.Module):
    def __init__(self, block, layers, num_channels=1, num_filter=7, num_classes=3):
        super(ResNet, self).__init__()
        self.in_channels = 16
        self.conv = conv3x3(num_channels, 16)
        self.bn = nn.BatchNorm2d(16)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 16, layers[0])
        self.layer2 = self.make_layer(block, 32, layers[1], 2)
        self.layer3 = self.make_layer(block, 64, layers[2], 2)
        self.avg_pool = nn.AvgPool2d(num_filter)
        self.fc = nn.Linear(64, num_classes)
        self.delta_T = nn.Linear(num_classes, num_classes, False)
        nn.init.zeros_(self.delta_T.weight)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)
    
    def forward(self, x, revision=False, plot_deltaT=False):
        correction = self.delta_T.weight
        if plot_deltaT:
            print("Corr", correction)
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        activate = nn.Sigmoid()
        out = activate(out)
        if revision:
            return out,correction
        else:
            return out
def estimate_tmatrix(num_classes, outputs, y_pred, average=True):
    """
    Function to estimate the transition matrix based on the noisy class posterior

    :param num_classes: Number of classes
    :param outputs: Outputs from the softmax function
    :param y_pred: Prediction values
    :param average: Average the probabilities for each layer P(Y^{~}=i | x)
    :return T: Estimated transition matrix
    """
    T = np.empty((num_classes, num_classes))
    # Iterate over the number of classes
    for i in range(num_classes):
        # Average the probability per column
        if average:
            # Get the samples with this predicted class
            # Get the samples of these indexes P(Y^=i | X=Xi)
            Xi_outputs = outputs[(y_pred == i)]

            T[i, :] = np.mean(Xi_outputs, axis=0)

        else:
            # Get the argmax
            idx = np.argmax(outputs[:, i])
            # Iterave over each label to fill the matrix
            for j in np.arange(num_classes):
                T[i, j] = outputs[idx, j]
    return T

In [11]:
torch.cuda.is_available()

True

In [12]:
torch.cuda.get_device_name(0)

'GeForce GTX 1060'

In [13]:
import torch.nn.functional as F
class CNN_2_layers(nn.Module):
    def __init__(self,input_shape_0,input_shape_1,in_channels,out_channels,mid_channel_0,
                 kernel_size_0,kernel_size_1,maxpool_size,num_class,tran_matrix,drop = 0):
        super(CNN_2_layers, self).__init__()
        self.tran_matrix = tran_matrix
        self.drop = drop
        # First Convolutional Layer
        self.conv0 = nn.Sequential(
            nn.Conv2d(
                in_channels = in_channels,    
                out_channels = mid_channel_0,  
                kernel_size = kernel_size_0, # Squar kernal
                stride = 1,
                padding = int((kernel_size_0 - 1)/2),
                groups = in_channels,        # Number of channel groups
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = maxpool_size)   # [n,*,28,28]
        )
        # First Fully Connection Layer
        self.fc_layer0 = nn.Sequential(
            nn.Linear(in_features = int(input_shape_1/(maxpool_size)), out_features = int(input_shape_1/(maxpool_size))),
            nn.ReLU()
        )
        ## Secound Convolutional Layer
        self.conv1 = nn.Sequential(
            nn.Conv2d(
                in_channels = mid_channel_0,    
                out_channels = out_channels,  
                kernel_size = kernel_size_1, # Squar kernal
                stride = 1,
                padding = int((kernel_size_1 - 1)/2),
                groups = mid_channel_0,         # Number of channel groups
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = maxpool_size)  # [n,*,28,28]
        )
        # Channel Aggregation Layer
        self.agg_layer = nn.Sequential(
            nn.Linear(in_features = out_channels, out_features = 1),
            nn.ReLU() # [n,1,28,28]
        )
        # Secound Fully Connection Layer
        self.fc_layer1 = nn.Sequential(
            nn.Linear(in_features = int(input_shape_1/((maxpool_size)**2)), out_features = 1),
            nn.ReLU()
        )
        # Third Fully Connection Layer
        self.fc_layer2 = nn.Sequential(
            nn.Linear(in_features = int(input_shape_0/((maxpool_size)**2)), out_features = num_class),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv0(x)
        x = self.fc_layer0(x)
        x = self.conv1(x)
        x = x.permute(0,2,3,1)
        x = self.agg_layer(x)
        x = torch.squeeze(x)
        x = self.fc_layer1(x)
        x = torch.squeeze(x)
        output0 = self.fc_layer2(x)
        output = torch.mm(self.tran_matrix,output0.T)
        output = output.T
        return output0,output

In [14]:
import torch.nn.functional as F
from torch.nn import Module

class ReweightingLoss(Module):
    def __init__(self):
      super(ReweightingLoss, self).__init__()
      
    def forward(self, out, T, target):
      loss = 0.0
      out_softmax = F.softmax(out, dim=1) 
      # If standard mode (revision false)
      g_Y = out_softmax.gather(1, target.view(-1,1))
      Tg = torch.mm(T.t(), out_softmax.t())
      Tg_Y = Tg.t().gather(1, target.view(-1,1))
      # requires_grad=True indicates that we want to compute gradients with
      # respect to these Variables during the backward pass.
      beta = torch.nn.Parameter(g_Y / Tg_Y)
      loss = F.cross_entropy(out, target, reduction='none')
      loss = beta.view(1,-1) * loss
      return torch.mean(loss)

class SoftBootstrappingLoss(Module):
    """
    ``Loss(t, p) = - (beta * t + (1 - beta) * p) * log(p)``
    Args:
        beta (float): bootstrap parameter. Default, 0.95
        reduce (bool): computes mean of the loss. Default, True.
        as_pseudo_label (bool): Stop gradient propagation for the term ``(1 - beta) * p``.
            Can be interpreted as pseudo-label.
    """
    def __init__(self, beta=0.1, reduce=True, as_pseudo_label=True):
        super(SoftBootstrappingLoss, self).__init__()
        self.beta = beta
        self.reduce = reduce
        self.as_pseudo_label = as_pseudo_label
        print('beta is ',beta)

    def forward(self, y_pred, y):

        # cross_entropy = - t * log(p)
        beta_xentropy = self.beta * F.cross_entropy(y_pred, y, reduction='none')

        y_pred_a = y_pred.detach() if self.as_pseudo_label else y_pred
        # second term = - (1 - beta) * p * log(p)
        bootstrap = - (1.0 - self.beta) * torch.sum(F.softmax(y_pred_a, dim=1) * F.log_softmax(y_pred, dim=1), dim=1)
        
        if self.reduce:
            return torch.mean(beta_xentropy + bootstrap)
        return beta_xentropy + bootstrap

class HardBootstrappingLoss(Module):
    """
    ``Loss(t, p) = - (beta * t + (1 - beta) * z) * log(p)``
    where ``z = argmax(p)``
    Args:
        beta (float): bootstrap parameter. Default, 0.95
        reduce (bool): computes mean of the loss. Default, True.
    """
    def __init__(self, beta=0.8, reduce=True):
        super(HardBootstrappingLoss, self).__init__()
        self.beta = beta
        self.reduce = reduce

    def forward(self, y_pred, y):
        # cross_entropy = - t * log(p)
        beta_xentropy = self.beta * F.cross_entropy(y_pred, y, reduction='none')

        # z = argmax(p)
        z = F.softmax(y_pred.detach(), dim=1).argmax(dim=1)
        z = z.view(-1, 1)
        bootstrap = F.log_softmax(y_pred, dim=1).gather(1, z).view(-1)
        # second term = (1 - beta) * z * log(p)
        bootstrap = - (1.0 - self.beta) * bootstrap

        if self.reduce:
            return torch.mean(beta_xentropy + bootstrap)
        return beta_xentropy + bootstrap

class TMT_loss(nn.Module):
    def __init__(self, beta = 1):
        super().__init__()
        self.beta = beta
        print('beta is ',self.beta)
        
    def forward(self, x, y, t):
        
        max_x = x.argmax(dim=1)
        new_x = x[max_x,:]
        now_t = torch.mm(y.T,new_x)/(x.size(0)/3)

        return self.beta*torch.mean(torch.abs(now_t - t)) + (1 - self.beta)*torch.mean(F.binary_cross_entropy(x,y,reduction='none'))

In [15]:
CIFAR = np.load('data/CIFAR.npz')
FMNIST5 = np.load('data/FashionMNIST5.npz')
FMNIST6 = np.load('data/FashionMNIST6.npz')

### FashionMNIST0.5

Data Processing

In [16]:
tran_matrix_FMNIST5 = torch.Tensor([[0.5,0.2,0.3],[0.3,0.5,0.2],[0.2,0.3,0.5]]).cuda()

labels = np.unique(FMNIST5['Str'])
num_label = len(labels)

X_test = torch.Tensor(FMNIST5['Xts']).unsqueeze(-1).cuda()/256 # Normalization
X_test.type(torch.float64)
Y_test_in = FMNIST5['Yts']

X_input = torch.Tensor(FMNIST5['Xtr']).unsqueeze(-1).cuda()/256 # Normalization
X_input.type(torch.float64)
Y_input = FMNIST5['Str']

tran_matrix = tran_matrix_FMNIST5
tran_matrix.requires_grad_(True)
num_img = X_input.size(0)

epoches = 1000
batch_size = 300
learning_rate = 1e-4

mid_channel_0 = 3
out_channels = 3
kernel_size_0 = 5
kernel_size_1 = 3
maxpool_size = 1
num_class = len(labels)

T estimate for FMNIST5

In [17]:
import torchvision.models as models

#torch.manual_seed(20)
num_img = X_input.size(0)

epoches = 3
batch_size = 300
learning_rate = 1e-4

mid_channel_0 = 3
out_channels = 3
kernel_size_0 = 5
kernel_size_1 = 3
maxpool_size = 1
num_class = len(labels)
print('Number of Class: ',num_class)
num_img = X_input.size(0)
idx_list = torch.randperm(num_img)
sep_point = int(num_img*0.8)
X_train = X_input[idx_list[:sep_point],:,:,:].permute(0,3,1,2)
Y_train_in = Y_input[idx_list[:sep_point]]
X_valid = X_input[idx_list[sep_point:],:,:,:].permute(0,3,1,2)
Y_valid_in = Y_input[idx_list[sep_point:]]
X_test = X_test.permute(0,3,1,2)

Y_test_exp = np.zeros([Y_test_in.shape[0],num_label],dtype = float)
Y_train_exp = np.zeros([Y_train_in.shape[0],num_label],dtype = float)
Y_valid_exp = np.zeros([Y_valid_in.shape[0],num_label],dtype = float)

for i in range(Y_train_exp.shape[0]):

    label_train_tmp = Y_train_in[i]
    Y_train_exp[i,label_train_tmp] = 1

for i in range(Y_test_exp.shape[0]):

    label_train_tmp = Y_test_in[i]
    Y_test_exp[i,label_train_tmp] = 1

for i in range(Y_valid_exp.shape[0]):

    label_train_tmp = Y_valid_in[i]
    Y_valid_exp[i,label_train_tmp] = 1

Y_train_exp = torch.Tensor(Y_train_exp).cuda()
Y_train = torch.Tensor(Y_train_in).cuda()
Y_train.type(torch.float64)
Y_test = torch.Tensor(Y_test_in)
Y_valid = torch.Tensor(Y_valid_in)

torch.manual_seed(10)

if X_train.size(1) == 1:
    num_filter = 7
else:
    num_filter = 8

t_estimator = ResNet(ResidualBlock, [2, 2, 2], num_channels = X_train.size(1), num_filter = num_filter,
                     num_classes = num_class).cuda()

train_data = Data.TensorDataset(X_train,Y_train_exp)
train_loader = Data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)

optimizer = torch.optim.Adam(t_estimator.parameters(), lr=learning_rate)
loss_function = nn.BCELoss() # Binary Cross Entropy

# Start Training T Estimator
for epoch in range(epoches):
    for step, (batch_x, batch_y) in enumerate(train_loader):
        output = t_estimator(batch_x)
        #print('output: ',output)
        loss = loss_function(output.cpu(), batch_y.cpu())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

t_estimator.cpu()
Y_nonoise_val = t_estimator(X_valid.cpu())
_,Y_nonoise_val = torch.max(Y_nonoise_val,1)
Y_nonoise_val = Y_nonoise_val.cpu()
accuracy = ((Y_nonoise_val == Y_valid).long().sum()) / float(Y_valid.size(0))

Y_nonoise_test = t_estimator(X_test.cpu())
_,Y_nonoise_test = torch.max(Y_nonoise_test,1)
Y_nonoise_test = Y_nonoise_test.cpu()
accuracy_test = ((Y_nonoise_test == Y_test).long().sum()) / float(Y_test.size(0))

print('Estimator  -->  Epoch: ', epoch, '| Step: ', step, '| train loss: %.4f' % torch.mean(loss).data.numpy(),
      '| Validation accuracy: %.2f' % accuracy, '| Testing accuracy: %.2f' % accuracy_test)
t_estimator.cuda()

t_estimator.cpu()
outputs = t_estimator(X_test.cpu())
estimated_T = estimate_tmatrix(num_classes = num_class, outputs = outputs.detach().numpy(), y_pred = Y_test)
print('\nEstimated T:\n',estimated_T)

Number of Class:  3
Estimator  -->  Epoch:  2 | Step:  47 | train loss: 0.5991 | Validation accuracy: 0.47 | Testing accuracy: 0.90

Estimated T:
 [[0.43472561 0.29026183 0.27093875]
 [0.21596934 0.48976886 0.30810452]
 [0.3078163  0.24223718 0.45377308]]


Data Seperating and Traning

In [18]:
test_acc_list_forward = []
test_acc_list_TMT = []

epoches = 1000
batch_size = 300
learning_rate = 1e-4

for the_index in range(10):
    
    # Forward Model
    torch.manual_seed(the_index*2)
    idx_list = torch.randperm(num_img)
    sep_point = int(num_img*0.8)
    X_train = X_input[idx_list[:sep_point],:,:,:].permute(0,3,1,2)
    Y_train_in = Y_input[idx_list[:sep_point]]
    X_valid = X_input[idx_list[sep_point:],:,:,:].permute(0,3,1,2)
    Y_valid_in = Y_input[idx_list[sep_point:]]

    Y_test_exp = np.zeros([Y_test_in.shape[0],num_label],dtype = float)
    Y_train_exp = np.zeros([Y_train_in.shape[0],num_label],dtype = float)
    Y_valid_exp = np.zeros([Y_valid_in.shape[0],num_label],dtype = float)

    for i in range(Y_train_exp.shape[0]):

        label_train_tmp = Y_train_in[i]
        Y_train_exp[i,label_train_tmp] = 1

    for i in range(Y_test_exp.shape[0]):

        label_train_tmp = Y_test_in[i]
        Y_test_exp[i,label_train_tmp] = 1

    for i in range(Y_valid_exp.shape[0]):

        label_train_tmp = Y_valid_in[i]
        Y_valid_exp[i,label_train_tmp] = 1

    Y_train_exp = torch.Tensor(Y_train_exp).cuda()
    Y_train = torch.Tensor(Y_train_in).cuda()
    Y_train.type(torch.float64)
    Y_test = torch.Tensor(Y_test_in)
    Y_valid = torch.Tensor(Y_valid_in)
    
    torch.manual_seed(1)
    train_data = Data.TensorDataset(X_train,Y_train_exp)
    train_loader = Data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)

    cnn = CNN_2_layers(X_train.size(2),X_train.size(3),X_train.size(1),out_channels,mid_channel_0,
              kernel_size_0,kernel_size_1,maxpool_size,num_class,tran_matrix).cuda()

    optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)
    loss_function = nn.BCELoss().cuda() # Binary Cross Entropy
    
    # Start Training Forward Model
    for epoch in range(epoches):
        for step, (batch_x, batch_y) in enumerate(train_loader):
            _,output = cnn(batch_x)
            loss = loss_function(output, batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if (epoch % 50 == 0):
            Y_nonoise_val,Y_pred = cnn(X_valid)
            _,Y_nonoise_val = torch.max(Y_nonoise_val,1)
            Y_nonoise_val = Y_nonoise_val.cpu()
            accuracy = ((Y_nonoise_val == Y_valid).long().sum()) / float(Y_valid.size(0))
            loss = loss.cpu()

            Y_nonoise_test,Y_pred_test = cnn(X_test)
            _,Y_nonoise_test = torch.max(Y_nonoise_test,1)
            Y_nonoise_test = Y_nonoise_test.cpu()
            accuracy_test = ((Y_nonoise_test == Y_test).long().sum()) / float(Y_test.size(0))
            

            print('Forward  -->  Epoch: ', epoch, '| Step: ', step, '| train loss: %.4f' % torch.mean(loss).data.numpy(),
                  '| Validation accuracy: %.2f' % accuracy, '| Testing accuracy: %.2f' % accuracy_test)
    test_acc_list_forward.append(accuracy_test)
    # TMT Model
    torch.manual_seed(the_index*2)
    idx_list = torch.randperm(num_img)
    sep_point = int(num_img*0.8)
    X_train = X_input[idx_list[:sep_point],:,:,:].permute(0,3,1,2)
    Y_train_in = Y_input[idx_list[:sep_point]]
    X_valid = X_input[idx_list[sep_point:],:,:,:].permute(0,3,1,2)
    Y_valid_in = Y_input[idx_list[sep_point:]]

    Y_test_exp = np.zeros([Y_test_in.shape[0],num_label],dtype = float)
    Y_train_exp = np.zeros([Y_train_in.shape[0],num_label],dtype = float)
    Y_valid_exp = np.zeros([Y_valid_in.shape[0],num_label],dtype = float)

    for i in range(Y_train_exp.shape[0]):

        label_train_tmp = Y_train_in[i]
        Y_train_exp[i,label_train_tmp] = 1

    for i in range(Y_test_exp.shape[0]):

        label_train_tmp = Y_test_in[i]
        Y_test_exp[i,label_train_tmp] = 1

    for i in range(Y_valid_exp.shape[0]):

        label_train_tmp = Y_valid_in[i]
        Y_valid_exp[i,label_train_tmp] = 1

    Y_train_exp = torch.Tensor(Y_train_exp).cuda()
    Y_train = torch.Tensor(Y_train_in).cuda()
    Y_train.type(torch.float64)
    Y_test = torch.Tensor(Y_test_in)
    Y_valid = torch.Tensor(Y_valid_in)

    torch.manual_seed(1)
    train_data = Data.TensorDataset(X_train,Y_train_exp)
    train_loader = Data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
    cnn_TMT = CNN_2_layers(X_train.size(2),X_train.size(3),X_train.size(1),out_channels,mid_channel_0,
                  kernel_size_0,kernel_size_1,maxpool_size,num_class,tran_matrix).cuda()

    optimizer_TMT = torch.optim.Adam(cnn_TMT.parameters(), lr=learning_rate)
    loss_function_TMT = TMT_loss(beta=0.2).cuda()
    
    # Start Training TMT Model
    for epoch in range(epoches):
        for step, (batch_x, batch_y) in enumerate(train_loader):

            output_TMT,_ = cnn_TMT(batch_x)
            loss_TMT = loss_function_TMT(output_TMT, batch_y,tran_matrix)
            optimizer_TMT.zero_grad()
            loss_TMT.backward()
            optimizer_TMT.step()

        if (epoch % 50 == 0):
            Y_nonoise_val,_ = cnn_TMT(X_valid)
            _,Y_nonoise_val = torch.max(Y_nonoise_val,1)
            Y_nonoise_val = Y_nonoise_val.cpu()
            accuracy = ((Y_nonoise_val == Y_valid).long().sum()) / float(Y_valid.size(0))
            loss_TMT = loss_TMT.cpu()

            Y_test_pre,_ = cnn_TMT(X_test)
            _,Y_test_pre = torch.max(Y_test_pre,1)
            Y_test_pre = Y_test_pre.cpu()
            accuracy_test = ((Y_test_pre == Y_test).long().sum()) / float(Y_test.size(0))

            print('TMT  -->  Epoch: ', epoch, '| Step: ', step, '| train loss: %.4f' % torch.mean(loss_TMT).data.numpy(),
                  '| Validation accuracy: %.2f' % accuracy, '| Testing accuracy: %.2f' % accuracy_test)
    test_acc_list_TMT.append(accuracy_test)

Forward  -->  Epoch:  0 | Step:  47 | train loss: 0.6695 | Validation accuracy: 0.33 | Testing accuracy: 0.33
Forward  -->  Epoch:  50 | Step:  47 | train loss: 0.6361 | Validation accuracy: 0.41 | Testing accuracy: 0.58
Forward  -->  Epoch:  100 | Step:  47 | train loss: 0.6243 | Validation accuracy: 0.48 | Testing accuracy: 0.86
Forward  -->  Epoch:  150 | Step:  47 | train loss: 0.6073 | Validation accuracy: 0.49 | Testing accuracy: 0.88
Forward  -->  Epoch:  200 | Step:  47 | train loss: 0.6239 | Validation accuracy: 0.50 | Testing accuracy: 0.89
Forward  -->  Epoch:  250 | Step:  47 | train loss: 0.6284 | Validation accuracy: 0.50 | Testing accuracy: 0.89
Forward  -->  Epoch:  300 | Step:  47 | train loss: 0.6172 | Validation accuracy: 0.50 | Testing accuracy: 0.90
Forward  -->  Epoch:  350 | Step:  47 | train loss: 0.6054 | Validation accuracy: 0.50 | Testing accuracy: 0.90
Forward  -->  Epoch:  400 | Step:  47 | train loss: 0.6213 | Validation accuracy: 0.50 | Testing accuracy: 

TMT  -->  Epoch:  750 | Step:  47 | train loss: 0.5049 | Validation accuracy: 0.47 | Testing accuracy: 0.90
TMT  -->  Epoch:  800 | Step:  47 | train loss: 0.5037 | Validation accuracy: 0.47 | Testing accuracy: 0.89
TMT  -->  Epoch:  850 | Step:  47 | train loss: 0.5205 | Validation accuracy: 0.47 | Testing accuracy: 0.89
TMT  -->  Epoch:  900 | Step:  47 | train loss: 0.5059 | Validation accuracy: 0.47 | Testing accuracy: 0.89
TMT  -->  Epoch:  950 | Step:  47 | train loss: 0.5232 | Validation accuracy: 0.47 | Testing accuracy: 0.89
Forward  -->  Epoch:  0 | Step:  47 | train loss: 0.6697 | Validation accuracy: 0.32 | Testing accuracy: 0.33
Forward  -->  Epoch:  50 | Step:  47 | train loss: 0.6364 | Validation accuracy: 0.35 | Testing accuracy: 0.43
Forward  -->  Epoch:  100 | Step:  47 | train loss: 0.6209 | Validation accuracy: 0.48 | Testing accuracy: 0.86
Forward  -->  Epoch:  150 | Step:  47 | train loss: 0.6132 | Validation accuracy: 0.48 | Testing accuracy: 0.89
Forward  -->  E

TMT  -->  Epoch:  500 | Step:  47 | train loss: 0.5005 | Validation accuracy: 0.48 | Testing accuracy: 0.90
TMT  -->  Epoch:  550 | Step:  47 | train loss: 0.5159 | Validation accuracy: 0.48 | Testing accuracy: 0.89
TMT  -->  Epoch:  600 | Step:  47 | train loss: 0.5133 | Validation accuracy: 0.48 | Testing accuracy: 0.90
TMT  -->  Epoch:  650 | Step:  47 | train loss: 0.5032 | Validation accuracy: 0.48 | Testing accuracy: 0.90
TMT  -->  Epoch:  700 | Step:  47 | train loss: 0.5025 | Validation accuracy: 0.48 | Testing accuracy: 0.90
TMT  -->  Epoch:  750 | Step:  47 | train loss: 0.5007 | Validation accuracy: 0.48 | Testing accuracy: 0.90
TMT  -->  Epoch:  800 | Step:  47 | train loss: 0.5098 | Validation accuracy: 0.48 | Testing accuracy: 0.89
TMT  -->  Epoch:  850 | Step:  47 | train loss: 0.4994 | Validation accuracy: 0.48 | Testing accuracy: 0.89
TMT  -->  Epoch:  900 | Step:  47 | train loss: 0.5025 | Validation accuracy: 0.48 | Testing accuracy: 0.89
TMT  -->  Epoch:  950 | Step

TMT  -->  Epoch:  250 | Step:  47 | train loss: 0.5122 | Validation accuracy: 0.48 | Testing accuracy: 0.89
TMT  -->  Epoch:  300 | Step:  47 | train loss: 0.5089 | Validation accuracy: 0.48 | Testing accuracy: 0.90
TMT  -->  Epoch:  350 | Step:  47 | train loss: 0.5182 | Validation accuracy: 0.48 | Testing accuracy: 0.90
TMT  -->  Epoch:  400 | Step:  47 | train loss: 0.5111 | Validation accuracy: 0.48 | Testing accuracy: 0.90
TMT  -->  Epoch:  450 | Step:  47 | train loss: 0.5098 | Validation accuracy: 0.48 | Testing accuracy: 0.90
TMT  -->  Epoch:  500 | Step:  47 | train loss: 0.5146 | Validation accuracy: 0.48 | Testing accuracy: 0.91
TMT  -->  Epoch:  550 | Step:  47 | train loss: 0.5047 | Validation accuracy: 0.48 | Testing accuracy: 0.91
TMT  -->  Epoch:  600 | Step:  47 | train loss: 0.5096 | Validation accuracy: 0.48 | Testing accuracy: 0.91
TMT  -->  Epoch:  650 | Step:  47 | train loss: 0.5025 | Validation accuracy: 0.48 | Testing accuracy: 0.91
TMT  -->  Epoch:  700 | Step

beta is  0.2
TMT  -->  Epoch:  0 | Step:  47 | train loss: 0.5693 | Validation accuracy: 0.32 | Testing accuracy: 0.33
TMT  -->  Epoch:  50 | Step:  47 | train loss: 0.5258 | Validation accuracy: 0.43 | Testing accuracy: 0.74
TMT  -->  Epoch:  100 | Step:  47 | train loss: 0.5086 | Validation accuracy: 0.46 | Testing accuracy: 0.85
TMT  -->  Epoch:  150 | Step:  47 | train loss: 0.5117 | Validation accuracy: 0.46 | Testing accuracy: 0.87
TMT  -->  Epoch:  200 | Step:  47 | train loss: 0.5060 | Validation accuracy: 0.47 | Testing accuracy: 0.88
TMT  -->  Epoch:  250 | Step:  47 | train loss: 0.5141 | Validation accuracy: 0.47 | Testing accuracy: 0.88
TMT  -->  Epoch:  300 | Step:  47 | train loss: 0.5145 | Validation accuracy: 0.47 | Testing accuracy: 0.89
TMT  -->  Epoch:  350 | Step:  47 | train loss: 0.5086 | Validation accuracy: 0.47 | Testing accuracy: 0.89
TMT  -->  Epoch:  400 | Step:  47 | train loss: 0.5179 | Validation accuracy: 0.47 | Testing accuracy: 0.89
TMT  -->  Epoch:  

Forward  -->  Epoch:  750 | Step:  47 | train loss: 0.5965 | Validation accuracy: 0.47 | Testing accuracy: 0.92
Forward  -->  Epoch:  800 | Step:  47 | train loss: 0.5948 | Validation accuracy: 0.47 | Testing accuracy: 0.92
Forward  -->  Epoch:  850 | Step:  47 | train loss: 0.5897 | Validation accuracy: 0.47 | Testing accuracy: 0.92
Forward  -->  Epoch:  900 | Step:  47 | train loss: 0.6114 | Validation accuracy: 0.47 | Testing accuracy: 0.92
Forward  -->  Epoch:  950 | Step:  47 | train loss: 0.5991 | Validation accuracy: 0.47 | Testing accuracy: 0.92
beta is  0.2
TMT  -->  Epoch:  0 | Step:  47 | train loss: 0.5681 | Validation accuracy: 0.33 | Testing accuracy: 0.33
TMT  -->  Epoch:  50 | Step:  47 | train loss: 0.5242 | Validation accuracy: 0.44 | Testing accuracy: 0.76
TMT  -->  Epoch:  100 | Step:  47 | train loss: 0.5130 | Validation accuracy: 0.46 | Testing accuracy: 0.86
TMT  -->  Epoch:  150 | Step:  47 | train loss: 0.5165 | Validation accuracy: 0.46 | Testing accuracy: 0.8

Result Output

In [19]:
print('test_acc_list_forward: ',test_acc_list_forward,'\nstander diviation and mean accuracy: ',torch.std_mean(torch.tensor(test_acc_list_forward)))
print('test_acc_list_forward: ',test_acc_list_TMT,'\nstander diviation and mean accuracy: ',torch.std_mean(torch.tensor(test_acc_list_TMT)))

test_acc_list_forward:  [tensor(0.9227), tensor(0.9123), tensor(0.9120), tensor(0.9163), tensor(0.9220), tensor(0.9133), tensor(0.9227), tensor(0.9180), tensor(0.9217), tensor(0.9183)] 
stander diviation and mean accuracy:  (tensor(0.0043), tensor(0.9179))
test_acc_list_forward:  [tensor(0.8983), tensor(0.8943), tensor(0.8897), tensor(0.8943), tensor(0.8953), tensor(0.9043), tensor(0.9027), tensor(0.9013), tensor(0.8973), tensor(0.8993)] 
stander diviation and mean accuracy:  (tensor(0.0044), tensor(0.8977))


### FashionMNIST0.6

Data Processing

In [20]:
tran_matrix_FMNIST6 = torch.Tensor([[0.4,0.3,0.3],[0.3,0.4,0.3],[0.3,0.3,0.4]]).cuda()

labels = np.unique(FMNIST6['Str'])
num_label = len(labels)

X_test = torch.Tensor(FMNIST6['Xts']).unsqueeze(-1).cuda()/256 # Normalization
X_test = X_test.permute(0,3,1,2)
X_test.type(torch.float64)
Y_test_in = FMNIST6['Yts']

X_input = torch.Tensor(FMNIST6['Xtr']).unsqueeze(-1).cuda()/256 # Normalization
X_input.type(torch.float64)
Y_input = FMNIST6['Str']

tran_matrix = tran_matrix_FMNIST6
tran_matrix.requires_grad_(True)
num_img = X_input.size(0)

epoches = 800
batch_size = 300
learning_rate = 1e-4

mid_channel_0 = 3
out_channels = 3
kernel_size_0 = 5
kernel_size_1 = 3
maxpool_size = 1
num_class = len(labels)

T estimate for FMNIST6

In [21]:
import torchvision.models as models

num_img = X_input.size(0)

epoches = 3
batch_size = 300
learning_rate = 1e-4

mid_channel_0 = 3
out_channels = 3
kernel_size_0 = 5
kernel_size_1 = 3
maxpool_size = 1
num_class = len(labels)
print('Number of Class: ',num_class)
idx_list = torch.randperm(num_img)
sep_point = int(num_img*0.8)
X_train = X_input[idx_list[:sep_point],:,:,:].permute(0,3,1,2)
Y_train_in = Y_input[idx_list[:sep_point]]
X_valid = X_input[idx_list[sep_point:],:,:,:].permute(0,3,1,2)
Y_valid_in = Y_input[idx_list[sep_point:]]

Y_test_exp = np.zeros([Y_test_in.shape[0],num_label],dtype = float)
Y_train_exp = np.zeros([Y_train_in.shape[0],num_label],dtype = float)
Y_valid_exp = np.zeros([Y_valid_in.shape[0],num_label],dtype = float)

for i in range(Y_train_exp.shape[0]):

    label_train_tmp = Y_train_in[i]
    Y_train_exp[i,label_train_tmp] = 1

for i in range(Y_test_exp.shape[0]):

    label_train_tmp = Y_test_in[i]
    Y_test_exp[i,label_train_tmp] = 1

for i in range(Y_valid_exp.shape[0]):

    label_train_tmp = Y_valid_in[i]
    Y_valid_exp[i,label_train_tmp] = 1

Y_train_exp = torch.Tensor(Y_train_exp).cuda()
Y_train = torch.Tensor(Y_train_in).cuda()
Y_train.type(torch.float64)
Y_test = torch.Tensor(Y_test_in)
Y_valid = torch.Tensor(Y_valid_in)

torch.manual_seed(10)

if X_train.size(1) == 1:
    num_filter = 7
else:
    num_filter = 8

t_estimator = ResNet(ResidualBlock, [2, 2, 2], num_channels = X_train.size(1), num_filter = num_filter,
                     num_classes = num_class).cuda()

train_data = Data.TensorDataset(X_train,Y_train_exp)
train_loader = Data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)

optimizer = torch.optim.Adam(t_estimator.parameters(), lr=learning_rate)
loss_function = nn.BCELoss() # Binary Cross Entropy

# Start Training T Estimator
for epoch in range(epoches):
    for step, (batch_x, batch_y) in enumerate(train_loader):
        output = t_estimator(batch_x)
        loss = loss_function(output.cpu(), batch_y.cpu())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

t_estimator.cpu()
Y_nonoise_val = t_estimator(X_valid.cpu())
_,Y_nonoise_val = torch.max(Y_nonoise_val,1)
Y_nonoise_val = Y_nonoise_val.cpu()
accuracy = ((Y_nonoise_val == Y_valid).long().sum()) / float(Y_valid.size(0))

Y_nonoise_test = t_estimator(X_test.cpu())
_,Y_nonoise_test = torch.max(Y_nonoise_test,1)
Y_nonoise_test = Y_nonoise_test.cpu()
accuracy_test = ((Y_nonoise_test == Y_test).long().sum()) / float(Y_test.size(0))

print('Estimator  -->  Epoch: ', epoch, '| Step: ', step, '| train loss: %.4f' % torch.mean(loss).data.numpy(),
      '| Validation accuracy: %.2f' % accuracy, '| Testing accuracy: %.2f' % accuracy_test)
t_estimator.cuda()

t_estimator.cpu()
outputs = t_estimator(X_test.cpu())
estimated_T = estimate_tmatrix(num_classes = num_class, outputs = outputs.detach().numpy(), y_pred = Y_test)
print('\nEstimated T:\n',estimated_T)

Number of Class:  3
Estimator  -->  Epoch:  2 | Step:  47 | train loss: 0.6362 | Validation accuracy: 0.37 | Testing accuracy: 0.79

Estimated T:
 [[0.38162607 0.30236644 0.31965521]
 [0.30174389 0.37130582 0.32806817]
 [0.30840048 0.30939433 0.38577226]]


Data Seperating and Traning

In [22]:
test_acc_list_forward = []
test_acc_list_TMT = []
epoches = 800
batch_size = 300
learning_rate = 1e-4
for the_index in range(10):
    
    # Forward Model
    torch.manual_seed(the_index*2)
    idx_list = torch.randperm(num_img)
    sep_point = int(num_img*0.8)
    X_train = X_input[idx_list[:sep_point],:,:,:].permute(0,3,1,2)
    Y_train_in = Y_input[idx_list[:sep_point]]
    X_valid = X_input[idx_list[sep_point:],:,:,:].permute(0,3,1,2)
    Y_valid_in = Y_input[idx_list[sep_point:]]

    Y_test_exp = np.zeros([Y_test_in.shape[0],num_label],dtype = float)
    Y_train_exp = np.zeros([Y_train_in.shape[0],num_label],dtype = float)
    Y_valid_exp = np.zeros([Y_valid_in.shape[0],num_label],dtype = float)

    for i in range(Y_train_exp.shape[0]):

        label_train_tmp = Y_train_in[i]
        Y_train_exp[i,label_train_tmp] = 1

    for i in range(Y_test_exp.shape[0]):

        label_train_tmp = Y_test_in[i]
        Y_test_exp[i,label_train_tmp] = 1

    for i in range(Y_valid_exp.shape[0]):

        label_train_tmp = Y_valid_in[i]
        Y_valid_exp[i,label_train_tmp] = 1

    Y_train_exp = torch.Tensor(Y_train_exp).cuda()
    Y_train = torch.Tensor(Y_train_in).cuda()
    Y_train.type(torch.float64)
    Y_test = torch.Tensor(Y_test_in)
    Y_valid = torch.Tensor(Y_valid_in)
    
    torch.manual_seed(1)
    train_data = Data.TensorDataset(X_train,Y_train_exp)
    train_loader = Data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)

    cnn = CNN_2_layers(X_train.size(2),X_train.size(3),X_train.size(1),out_channels,mid_channel_0,
              kernel_size_0,kernel_size_1,maxpool_size,num_class,tran_matrix).cuda()

    optimizer = torch.optim.Adam(cnn.parameters(), lr=learning_rate)
    loss_function = nn.BCELoss().cuda() # Binary Cross Entropy
    
    # Start Training Forward Model
    for epoch in range(epoches):
        for step, (batch_x, batch_y) in enumerate(train_loader):
            _,output = cnn(batch_x)
            loss = loss_function(output, batch_y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        if (epoch % 50 == 0):
            Y_nonoise_val,Y_pred = cnn(X_valid)
            _,Y_nonoise_val = torch.max(Y_nonoise_val,1)
            Y_nonoise_val = Y_nonoise_val.cpu()
            accuracy = ((Y_nonoise_val == Y_valid).long().sum()) / float(Y_valid.size(0))
            loss = loss.cpu()

            Y_nonoise_test,Y_pred_test = cnn(X_test)
            _,Y_nonoise_test = torch.max(Y_nonoise_test,1)
            Y_nonoise_test = Y_nonoise_test.cpu()
            accuracy_test = ((Y_nonoise_test == Y_test).long().sum()) / float(Y_test.size(0))

            print('Forward  -->  Epoch: ', epoch, '| Step: ', step, '| train loss: %.4f' % torch.mean(loss).data.numpy(),
                  '| Validation accuracy: %.2f' % accuracy, '| Testing accuracy: %.2f' % accuracy_test)
    test_acc_list_forward.append(accuracy_test)

    # TMT Model
    torch.manual_seed(the_index*2)
    idx_list = torch.randperm(num_img)
    sep_point = int(num_img*0.8)
    X_train = X_input[idx_list[:sep_point],:,:,:].permute(0,3,1,2)
    Y_train_in = Y_input[idx_list[:sep_point]]
    X_valid = X_input[idx_list[sep_point:],:,:,:].permute(0,3,1,2)
    Y_valid_in = Y_input[idx_list[sep_point:]]

    Y_test_exp = np.zeros([Y_test_in.shape[0],num_label],dtype = float)
    Y_train_exp = np.zeros([Y_train_in.shape[0],num_label],dtype = float)
    Y_valid_exp = np.zeros([Y_valid_in.shape[0],num_label],dtype = float)

    for i in range(Y_train_exp.shape[0]):

        label_train_tmp = Y_train_in[i]
        Y_train_exp[i,label_train_tmp] = 1

    for i in range(Y_test_exp.shape[0]):

        label_train_tmp = Y_test_in[i]
        Y_test_exp[i,label_train_tmp] = 1

    for i in range(Y_valid_exp.shape[0]):

        label_train_tmp = Y_valid_in[i]
        Y_valid_exp[i,label_train_tmp] = 1

    Y_train_exp = torch.Tensor(Y_train_exp).cuda()
    Y_train = torch.Tensor(Y_train_in).cuda()
    Y_train.type(torch.float64)
    Y_test = torch.Tensor(Y_test_in)
    Y_valid = torch.Tensor(Y_valid_in)

    torch.manual_seed(1)
    train_data = Data.TensorDataset(X_train,Y_train_exp)
    train_loader = Data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
    cnn_TMT = CNN_2_layers(X_train.size(2),X_train.size(3),X_train.size(1),out_channels,mid_channel_0,
                  kernel_size_0,kernel_size_1,maxpool_size,num_class,tran_matrix).cuda()

    optimizer_TMT = torch.optim.Adam(cnn_TMT.parameters(), lr=learning_rate)
    loss_function_TMT = TMT_loss(beta=0.2).cuda()
    
    # Start Training TMT Model
    for epoch in range(epoches):
        for step, (batch_x, batch_y) in enumerate(train_loader):

            output_TMT,_ = cnn_TMT(batch_x)
            loss_TMT = loss_function_TMT(output_TMT, batch_y,tran_matrix)
            optimizer_TMT.zero_grad()
            loss_TMT.backward()
            optimizer_TMT.step()

        if (epoch % 50 == 0):
            Y_nonoise_val,_ = cnn_TMT(X_valid)
            _,Y_nonoise_val = torch.max(Y_nonoise_val,1)
            Y_nonoise_val = Y_nonoise_val.cpu()
            accuracy = ((Y_nonoise_val == Y_valid).long().sum()) / float(Y_valid.size(0))
            loss_TMT = loss_TMT.cpu()

            Y_test_pre,_ = cnn_TMT(X_test)
            _,Y_test_pre = torch.max(Y_test_pre,1)
            Y_test_pre = Y_test_pre.cpu()
            accuracy_test = ((Y_test_pre == Y_test).long().sum()) / float(Y_test.size(0))

            print('TMT  -->  Epoch: ', epoch, '| Step: ', step, '| train loss: %.4f' % torch.mean(loss_TMT).data.numpy(),
                  '| Validation accuracy: %.2f' % accuracy, '| Testing accuracy: %.2f' % accuracy_test)
    test_acc_list_TMT.append(accuracy_test)

Forward  -->  Epoch:  0 | Step:  47 | train loss: 0.6692 | Validation accuracy: 0.33 | Testing accuracy: 0.33
Forward  -->  Epoch:  50 | Step:  47 | train loss: 0.6363 | Validation accuracy: 0.33 | Testing accuracy: 0.33
Forward  -->  Epoch:  100 | Step:  47 | train loss: 0.6365 | Validation accuracy: 0.33 | Testing accuracy: 0.33
Forward  -->  Epoch:  150 | Step:  47 | train loss: 0.6364 | Validation accuracy: 0.33 | Testing accuracy: 0.34
Forward  -->  Epoch:  200 | Step:  47 | train loss: 0.6357 | Validation accuracy: 0.39 | Testing accuracy: 0.76
Forward  -->  Epoch:  250 | Step:  47 | train loss: 0.6329 | Validation accuracy: 0.41 | Testing accuracy: 0.85
Forward  -->  Epoch:  300 | Step:  47 | train loss: 0.6336 | Validation accuracy: 0.40 | Testing accuracy: 0.86
Forward  -->  Epoch:  350 | Step:  47 | train loss: 0.6320 | Validation accuracy: 0.40 | Testing accuracy: 0.86
Forward  -->  Epoch:  400 | Step:  47 | train loss: 0.6343 | Validation accuracy: 0.40 | Testing accuracy: 

Forward  -->  Epoch:  550 | Step:  47 | train loss: 0.6314 | Validation accuracy: 0.39 | Testing accuracy: 0.86
Forward  -->  Epoch:  600 | Step:  47 | train loss: 0.6309 | Validation accuracy: 0.39 | Testing accuracy: 0.86
Forward  -->  Epoch:  650 | Step:  47 | train loss: 0.6346 | Validation accuracy: 0.39 | Testing accuracy: 0.86
Forward  -->  Epoch:  700 | Step:  47 | train loss: 0.6281 | Validation accuracy: 0.39 | Testing accuracy: 0.86
Forward  -->  Epoch:  750 | Step:  47 | train loss: 0.6330 | Validation accuracy: 0.39 | Testing accuracy: 0.86
beta is  0.2
TMT  -->  Epoch:  0 | Step:  47 | train loss: 0.5614 | Validation accuracy: 0.33 | Testing accuracy: 0.33
TMT  -->  Epoch:  50 | Step:  47 | train loss: 0.5205 | Validation accuracy: 0.35 | Testing accuracy: 0.65
TMT  -->  Epoch:  100 | Step:  47 | train loss: 0.5169 | Validation accuracy: 0.36 | Testing accuracy: 0.72
TMT  -->  Epoch:  150 | Step:  47 | train loss: 0.5158 | Validation accuracy: 0.36 | Testing accuracy: 0.7

TMT  -->  Epoch:  300 | Step:  47 | train loss: 0.5184 | Validation accuracy: 0.39 | Testing accuracy: 0.81
TMT  -->  Epoch:  350 | Step:  47 | train loss: 0.5181 | Validation accuracy: 0.39 | Testing accuracy: 0.81
TMT  -->  Epoch:  400 | Step:  47 | train loss: 0.5151 | Validation accuracy: 0.40 | Testing accuracy: 0.83
TMT  -->  Epoch:  450 | Step:  47 | train loss: 0.5101 | Validation accuracy: 0.40 | Testing accuracy: 0.83
TMT  -->  Epoch:  500 | Step:  47 | train loss: 0.5119 | Validation accuracy: 0.40 | Testing accuracy: 0.83
TMT  -->  Epoch:  550 | Step:  47 | train loss: 0.5147 | Validation accuracy: 0.40 | Testing accuracy: 0.83
TMT  -->  Epoch:  600 | Step:  47 | train loss: 0.5142 | Validation accuracy: 0.40 | Testing accuracy: 0.83
TMT  -->  Epoch:  650 | Step:  47 | train loss: 0.5159 | Validation accuracy: 0.40 | Testing accuracy: 0.82
TMT  -->  Epoch:  700 | Step:  47 | train loss: 0.5156 | Validation accuracy: 0.40 | Testing accuracy: 0.82
TMT  -->  Epoch:  750 | Step

Forward  -->  Epoch:  50 | Step:  47 | train loss: 0.6365 | Validation accuracy: 0.33 | Testing accuracy: 0.33
Forward  -->  Epoch:  100 | Step:  47 | train loss: 0.6365 | Validation accuracy: 0.36 | Testing accuracy: 0.52
Forward  -->  Epoch:  150 | Step:  47 | train loss: 0.6364 | Validation accuracy: 0.36 | Testing accuracy: 0.54
Forward  -->  Epoch:  200 | Step:  47 | train loss: 0.6350 | Validation accuracy: 0.38 | Testing accuracy: 0.79
Forward  -->  Epoch:  250 | Step:  47 | train loss: 0.6373 | Validation accuracy: 0.39 | Testing accuracy: 0.85
Forward  -->  Epoch:  300 | Step:  47 | train loss: 0.6336 | Validation accuracy: 0.39 | Testing accuracy: 0.85
Forward  -->  Epoch:  350 | Step:  47 | train loss: 0.6306 | Validation accuracy: 0.39 | Testing accuracy: 0.85
Forward  -->  Epoch:  400 | Step:  47 | train loss: 0.6334 | Validation accuracy: 0.39 | Testing accuracy: 0.85
Forward  -->  Epoch:  450 | Step:  47 | train loss: 0.6331 | Validation accuracy: 0.39 | Testing accuracy

Forward  -->  Epoch:  600 | Step:  47 | train loss: 0.6333 | Validation accuracy: 0.39 | Testing accuracy: 0.86
Forward  -->  Epoch:  650 | Step:  47 | train loss: 0.6305 | Validation accuracy: 0.39 | Testing accuracy: 0.86
Forward  -->  Epoch:  700 | Step:  47 | train loss: 0.6389 | Validation accuracy: 0.39 | Testing accuracy: 0.86
Forward  -->  Epoch:  750 | Step:  47 | train loss: 0.6334 | Validation accuracy: 0.39 | Testing accuracy: 0.86
beta is  0.2
TMT  -->  Epoch:  0 | Step:  47 | train loss: 0.5605 | Validation accuracy: 0.33 | Testing accuracy: 0.33
TMT  -->  Epoch:  50 | Step:  47 | train loss: 0.5177 | Validation accuracy: 0.36 | Testing accuracy: 0.55
TMT  -->  Epoch:  100 | Step:  47 | train loss: 0.5170 | Validation accuracy: 0.38 | Testing accuracy: 0.76
TMT  -->  Epoch:  150 | Step:  47 | train loss: 0.5162 | Validation accuracy: 0.39 | Testing accuracy: 0.81
TMT  -->  Epoch:  200 | Step:  47 | train loss: 0.5134 | Validation accuracy: 0.39 | Testing accuracy: 0.83
TM

Result Output

In [23]:
print('test_acc_list_forward: ',test_acc_list_forward,'\nstander diviation and mean accuracy: ',torch.std_mean(torch.tensor(test_acc_list_forward)))
print('test_acc_list_forward: ',test_acc_list_TMT,'\nstander diviation and mean accuracy: ',torch.std_mean(torch.tensor(test_acc_list_TMT)))

test_acc_list_forward:  [tensor(0.8643), tensor(0.8473), tensor(0.8590), tensor(0.8657), tensor(0.8807), tensor(0.8583), tensor(0.8520), tensor(0.8520), tensor(0.8590), tensor(0.8567)] 
stander diviation and mean accuracy:  (tensor(0.0093), tensor(0.8595))
test_acc_list_forward:  [tensor(0.8400), tensor(0.7863), tensor(0.8040), tensor(0.8050), tensor(0.8177), tensor(0.8093), tensor(0.8033), tensor(0.8093), tensor(0.8130), tensor(0.7933)] 
stander diviation and mean accuracy:  (tensor(0.0144), tensor(0.8081))


### CIFAR

RestNet without T matrix

In [16]:
import torchvision.models as models

labels = np.unique(CIFAR['Str'])
num_label = len(labels)

X_test = torch.Tensor(CIFAR['Xts']).cuda()/256 # Normalization
X_test = X_test.permute(0,3,1,2)
X_test.type(torch.float64)
Y_test_in = CIFAR['Yts']

X_input = torch.Tensor(CIFAR['Xtr']).cuda()/256 # Normalization
X_input.type(torch.float64)
Y_input = CIFAR['Str']

num_img = X_input.size(0)

epoches = 3
batch_size = 100
learning_rate = 1e-4

mid_channel_0 = 3
out_channels = 3
kernel_size_0 = 5
kernel_size_1 = 3
maxpool_size = 1
num_class = len(labels)
print('Number of Class: ',num_class)
acc_list = []
tran_mt_list = []

for the_index in range(10):
    idx_list = torch.randperm(num_img)
    sep_point = int(num_img*0.8)
    X_train = X_input[idx_list[:sep_point],:,:,:].permute(0,3,1,2)
    Y_train_in = Y_input[idx_list[:sep_point]]
    X_valid = X_input[idx_list[sep_point:],:,:,:].permute(0,3,1,2)
    Y_valid_in = Y_input[idx_list[sep_point:]]

    Y_test_exp = np.zeros([Y_test_in.shape[0],num_label],dtype = float)
    Y_train_exp = np.zeros([Y_train_in.shape[0],num_label],dtype = float)
    Y_valid_exp = np.zeros([Y_valid_in.shape[0],num_label],dtype = float)

    for i in range(Y_train_exp.shape[0]):

        label_train_tmp = Y_train_in[i]
        Y_train_exp[i,label_train_tmp] = 1

    for i in range(Y_test_exp.shape[0]):

        label_train_tmp = Y_test_in[i]
        Y_test_exp[i,label_train_tmp] = 1

    for i in range(Y_valid_exp.shape[0]):

        label_train_tmp = Y_valid_in[i]
        Y_valid_exp[i,label_train_tmp] = 1

    Y_train_exp = torch.Tensor(Y_train_exp).cuda()
    Y_train = torch.Tensor(Y_train_in).cuda()
    Y_train.type(torch.float64)
    Y_test = torch.Tensor(Y_test_in)
    Y_valid = torch.Tensor(Y_valid_in)
    torch.manual_seed(10) # Set Random Seed

    if X_train.size(1) == 1:
        num_filter = 7
    else:
        num_filter = 8

    t_estimator = ResNet(ResidualBlock, [2, 2, 2], num_channels = X_train.size(1), num_filter = num_filter, num_classes = num_class).cuda()

    train_data = Data.TensorDataset(X_train,Y_train_exp)
    train_loader = Data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)

    optimizer = torch.optim.Adam(t_estimator.parameters(), lr=learning_rate)
    loss_function = nn.BCELoss() # Binary Cross Entropy

    # Start Training T Estimator
    for epoch in range(epoches):
        for step, (batch_x, batch_y) in enumerate(train_loader):
            output = t_estimator(batch_x)
            #print('output: ',output)
            loss = loss_function(output.cpu(), batch_y.cpu())
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    t_estimator.cpu()
    Y_nonoise_val = t_estimator(X_valid.cpu())
    #print('Now T: ',tran_matrix)
    _,Y_nonoise_val = torch.max(Y_nonoise_val,1)
    Y_nonoise_val = Y_nonoise_val.cpu()
    accuracy = ((Y_nonoise_val == Y_valid).long().sum()) / float(Y_valid.size(0))
    #loss = loss.cpu()

    Y_nonoise_test = t_estimator(X_test.cpu())
    _,Y_nonoise_test = torch.max(Y_nonoise_test,1)
    Y_nonoise_test = Y_nonoise_test.cpu()
    accuracy_test = ((Y_nonoise_test == Y_test).long().sum()) / float(Y_test.size(0))

    print('Estimator  -->  Epoch: ', epoch, '| Step: ', step, '| train loss: %.4f' % torch.mean(loss).data.numpy(),
          '| Validation accuracy: %.2f' % accuracy, '| Testing accuracy: %.2f' % accuracy_test)
    acc_list.append(accuracy_test)
    t_estimator.cuda()

    t_estimator.cpu()
    outputs = t_estimator(X_test.cpu())
    estimated_T = estimate_tmatrix(num_classes = num_class, outputs = outputs.detach().numpy(), y_pred = Y_test)
    print(f"Estimated Transition Matrix: \n{np.round(estimated_T, 4)}")
    tran_mt_list.append(np.round(estimated_T, 4))

Number of Class:  3
Estimator  -->  Epoch:  2 | Step:  119 | train loss: 0.6441 | Validation accuracy: 0.37 | Testing accuracy: 0.53
Estimated Transition Matrix: 
[[0.3661 0.3466 0.2972]
 [0.2875 0.3677 0.355 ]
 [0.1874 0.3979 0.4822]]
Estimator  -->  Epoch:  2 | Step:  119 | train loss: 0.6482 | Validation accuracy: 0.37 | Testing accuracy: 0.70
Estimated Transition Matrix: 
[[0.3718 0.3421 0.2815]
 [0.3179 0.3702 0.3108]
 [0.3271 0.2588 0.431 ]]
Estimator  -->  Epoch:  2 | Step:  119 | train loss: 0.6479 | Validation accuracy: 0.38 | Testing accuracy: 0.69
Estimated Transition Matrix: 
[[0.3716 0.3429 0.2816]
 [0.3183 0.3712 0.3104]
 [0.3266 0.2628 0.43  ]]
Estimator  -->  Epoch:  2 | Step:  119 | train loss: 0.6470 | Validation accuracy: 0.38 | Testing accuracy: 0.69
Estimated Transition Matrix: 
[[0.3717 0.3434 0.2814]
 [0.3179 0.37   0.3099]
 [0.3311 0.2612 0.4276]]
Estimator  -->  Epoch:  2 | Step:  119 | train loss: 0.6472 | Validation accuracy: 0.38 | Testing accuracy: 0.69
Est

In [17]:
print('acc_list: ',acc_list,'\nstander diviation and mean accuracy: ',torch.std_mean(torch.tensor(acc_list)))
all_mt = []
for i in range(len(tran_mt_list)):
    if i == 0:
        all_mt = tran_mt_list[i]
    else:
        all_mt = all_mt + tran_mt_list[i]
avg_mt = all_mt/len(tran_mt_list)
print('tran_mt: ',avg_mt)

acc_list:  [tensor(0.5347), tensor(0.6960), tensor(0.6907), tensor(0.6870), tensor(0.6947), tensor(0.6930), tensor(0.6887), tensor(0.6847), tensor(0.6967), tensor(0.6943)] 
stander diviation and mean accuracy:  (tensor(0.0498), tensor(0.6760))
tran_mt:  [[0.37008 0.34347 0.28305]
 [0.31489 0.36983 0.31504]
 [0.3129  0.2726  0.43454]]


Resnet With T Matrix

In [24]:
tran_matrix = torch.tensor(avg_mt).cuda()
tran_matrix.requires_grad_(True)
labels = np.unique(CIFAR['Str'])
num_label = len(labels)

X_test = torch.Tensor(CIFAR['Xts']).cuda()/256 # Normalization
X_test = X_test.permute(0,3,1,2)
X_test.type(torch.float64)
Y_test_in = CIFAR['Yts']

X_input = torch.Tensor(CIFAR['Xtr']).cuda()/256 # Normalization
X_input.type(torch.float64)
Y_input = CIFAR['Str']

num_img = X_input.size(0)

epoches = 3
batch_size = 100
learning_rate = 1e-4

mid_channel_0 = 3
out_channels = 3
kernel_size_0 = 5
kernel_size_1 = 3
maxpool_size = 1
num_class = len(labels)
print('Number of Class: ',num_class)
acc_list = []

for the_index in range(10):
    idx_list = torch.randperm(num_img)
    sep_point = int(num_img*0.8)
    X_train = X_input[idx_list[:sep_point],:,:,:].permute(0,3,1,2)
    Y_train_in = Y_input[idx_list[:sep_point]]
    X_valid = X_input[idx_list[sep_point:],:,:,:].permute(0,3,1,2)
    Y_valid_in = Y_input[idx_list[sep_point:]]

    Y_test_exp = np.zeros([Y_test_in.shape[0],num_label],dtype = float)
    Y_train_exp = np.zeros([Y_train_in.shape[0],num_label],dtype = float)
    Y_valid_exp = np.zeros([Y_valid_in.shape[0],num_label],dtype = float)

    for i in range(Y_train_exp.shape[0]):

        label_train_tmp = Y_train_in[i]
        Y_train_exp[i,label_train_tmp] = 1

    for i in range(Y_test_exp.shape[0]):

        label_train_tmp = Y_test_in[i]
        Y_test_exp[i,label_train_tmp] = 1

    for i in range(Y_valid_exp.shape[0]):

        label_train_tmp = Y_valid_in[i]
        Y_valid_exp[i,label_train_tmp] = 1

    Y_train_exp = torch.Tensor(Y_train_exp).cuda()
    Y_train = torch.Tensor(Y_train_in).cuda()
    Y_train.type(torch.float64)
    Y_test = torch.Tensor(Y_test_in)
    Y_valid = torch.Tensor(Y_valid_in)

    torch.manual_seed(10)

    if X_train.size(1) == 1:
        num_filter = 7
    else:
        num_filter = 8

    resnet = ResNet(ResidualBlock, [2, 2, 2], num_channels = X_train.size(1), num_filter = num_filter, num_classes = num_class).cuda()

    train_data = Data.TensorDataset(X_train,Y_train_exp)
    train_loader = Data.DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)

    optimizer = torch.optim.Adam(resnet.parameters(), lr=learning_rate)
    loss_function = TMT_loss(beta=0.2).cuda()

    # Start Training TMT_loss Classifier
    for epoch in range(epoches):
        for step, (batch_x, batch_y) in enumerate(train_loader):
            output = resnet(batch_x)
            #print('output: ',output)
            loss = loss_function(output, batch_y,tran_matrix)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    resnet.cpu()
    Y_nonoise_val = resnet(X_valid.cpu())
    _,Y_nonoise_val = torch.max(Y_nonoise_val,1)
    Y_nonoise_val = Y_nonoise_val.cpu()
    accuracy = ((Y_nonoise_val == Y_valid).long().sum()) / float(Y_valid.size(0))

    Y_nonoise_test = resnet(X_test.cpu())
    _,Y_nonoise_test = torch.max(Y_nonoise_test,1)
    Y_nonoise_test = Y_nonoise_test.cpu()
    accuracy_test = ((Y_nonoise_test == Y_test).long().sum()) / float(Y_test.size(0))

    print('Estimator  -->  Epoch: ', epoch, '| Step: ', step, '| train loss: %.4f' % torch.mean(loss).data.cpu().numpy(),
          '| Validation accuracy: %.2f' % accuracy, '| Testing accuracy: %.2f' % accuracy_test)
    acc_list.append(accuracy_test)
    resnet.cuda()

Number of Class:  3
beta is  0.2
Estimator  -->  Epoch:  2 | Step:  119 | train loss: 0.5409 | Validation accuracy: 0.37 | Testing accuracy: 0.69
beta is  0.2
Estimator  -->  Epoch:  2 | Step:  119 | train loss: 0.5619 | Validation accuracy: 0.37 | Testing accuracy: 0.69
beta is  0.2
Estimator  -->  Epoch:  2 | Step:  119 | train loss: 0.5617 | Validation accuracy: 0.37 | Testing accuracy: 0.69
beta is  0.2
Estimator  -->  Epoch:  2 | Step:  119 | train loss: 0.5616 | Validation accuracy: 0.38 | Testing accuracy: 0.69
beta is  0.2
Estimator  -->  Epoch:  2 | Step:  119 | train loss: 0.5617 | Validation accuracy: 0.37 | Testing accuracy: 0.69
beta is  0.2
Estimator  -->  Epoch:  2 | Step:  119 | train loss: 0.5616 | Validation accuracy: 0.37 | Testing accuracy: 0.69
beta is  0.2
Estimator  -->  Epoch:  2 | Step:  119 | train loss: 0.5619 | Validation accuracy: 0.38 | Testing accuracy: 0.69
beta is  0.2
Estimator  -->  Epoch:  2 | Step:  119 | train loss: 0.5615 | Validation accuracy: 0.

In [25]:
print('acc_list: ',acc_list,'\nstander diviation and mean accuracy: ',torch.std_mean(torch.tensor(acc_list)))

acc_list:  [tensor(0.6927), tensor(0.6937), tensor(0.6930), tensor(0.6850), tensor(0.6933), tensor(0.6950), tensor(0.6883), tensor(0.6883), tensor(0.6920), tensor(0.6807)] 
stander diviation and mean accuracy:  (tensor(0.0046), tensor(0.6902))


### Baseline
Our Baseline model is the CNN of keras

In [20]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import SGD

def create_CNN(kernel_list = [5,5],stride_list = [1,1],learn_rate = 1e-3):

    CNN = Sequential()
    CNN.add(Conv2D(32, kernel_size = kernel_list, strides = stride_list))
    CNN.add(MaxPooling2D())
    CNN.add(Flatten())
    CNN.add(Dense(500,activation = 'relu'))
    CNN.add(Dense(3, activation = 'softmax'))
    optimizer = SGD(lr=learn_rate)
    CNN.compile(loss='binary_crossentropy',optimizer=optimizer,metrics= ['accuracy'])
    print("kernel_list: ",kernel_list,"\nstride_list: ",stride_list, "\nlearn_rate: ",learn_rate)
    return CNN

Using TensorFlow backend.


CIFAR

In [21]:
X_train = X_train.permute(0,2,3,1).cpu().numpy()
X_test = X_test.permute(0,2,3,1).cpu().numpy()
X_valid = X_valid.permute(0,2,3,1).cpu().numpy()
Y_train_exp = Y_train_exp.cpu().numpy()
Y_test = Y_test.cpu().numpy()
Y_valid = Y_valid.cpu().numpy()

print("X_train: ",np.shape(X_train),"\nY_train: ",np.shape(Y_train),"\nX_test: ",np.shape(X_test),"\nY_test: ",np.shape(Y_test_exp),
      "\nX_valid: ",np.shape(X_valid),"\nY_valid: ",np.shape(Y_valid_exp))

CNN = create_CNN()

CNN.fit(X_train,Y_train_exp,batch_size=300,epochs=5)

Y_pred = CNN.predict(X_test,batch_size=300)
Y_pred = np.argmax(Y_pred, axis=1)
count = 0
for i in range(len(Y_pred)):
    if Y_pred[i] == Y_test[i]:
        count += 1
print(count)
accuracy = count/np.shape(Y_test)[0]
print('Test Accuracy: ',accuracy)

Y_pred = CNN.predict(X_valid,batch_size=300)
Y_pred = np.argmax(Y_pred, axis=1)
count = 0
for i in range(len(Y_pred)):
    if Y_pred[i] == Y_valid[i]:
        count += 1
print(count)
accuracy = count/np.shape(Y_valid)[0]
print('Validation Accuracy: ',accuracy)

X_train:  (12000, 32, 32, 3) 
Y_train:  torch.Size([12000]) 
X_test:  (3000, 32, 32, 3) 
Y_test:  (3000, 3) 
X_valid:  (3000, 32, 32, 3) 
Y_valid:  (3000, 3)


kernel_list:  [5, 5] 
stride_list:  [1, 1] 
learn_rate:  0.001




Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Epoch 1/5





12000/12000 [==============================] - 5s 419us/step - loss: 0.6390 - acc: 0.6667
Epoch 2/5
12000/12000 [==============================] - 5s 384us/step - loss: 0.6385 - acc: 0.6667
Epoch 3/5
12000/12000 [==============================] - 5s 389us/step - loss: 0.6381 - acc: 0.6667
Epoch 4/5
12000/12000 [==============================] - 5s 379us/step - loss: 0.6378 - acc: 0.6667
Epoch 5/5
12000/12000 [==============================] - 5s 381us/step - loss: 0.6374 - acc: 0.6667
1056
Test Accuracy:  0.352
1014
Validation Accuracy:  0.338


FMNIST5

In [22]:
labels = np.unique(FMNIST5['Str'])
num_label = len(labels)

X_test = torch.Tensor(FMNIST5['Xts']).unsqueeze(-1)/256 # Normalization
X_test.type(torch.float64)
Y_test_in = FMNIST5['Yts']

X_input = torch.Tensor(FMNIST5['Xtr']).unsqueeze(-1)/256 # Normalization
X_input.type(torch.float64)
Y_input = FMNIST5['Str']

num_img = X_input.size(0)
idx_list = torch.randperm(num_img)
sep_point = int(num_img*0.8)
X_train = X_input[idx_list[:sep_point],:,:,:].permute(0,3,1,2)
Y_train_in = Y_input[idx_list[:sep_point]]
X_valid = X_input[idx_list[sep_point:],:,:,:].permute(0,3,1,2)
Y_valid_in = Y_input[idx_list[sep_point:]]
X_test = X_test.permute(0,3,1,2)

Y_test_exp = np.zeros([Y_test_in.shape[0],num_label],dtype = float)
Y_train_exp = np.zeros([Y_train_in.shape[0],num_label],dtype = float)

for i in range(Y_train_exp.shape[0]):

    label_train_tmp = Y_train_in[i]
    Y_train_exp[i,label_train_tmp] = 1

for i in range(Y_test_exp.shape[0]):

    label_train_tmp = Y_test_in[i]
    Y_test_exp[i,label_train_tmp] = 1

X_train = X_train.permute(0,2,3,1).cpu().numpy()
X_test = X_test.permute(0,2,3,1).cpu().numpy()
X_valid = X_valid.permute(0,2,3,1).cpu().numpy()
Y_test = np.array(Y_test_in)
Y_valid = np.array(Y_valid_in)


CNN = create_CNN()

CNN.fit(X_train,Y_train_exp,batch_size=300,epochs=5)

Y_pred = CNN.predict(X_test,batch_size=300)
Y_pred = np.argmax(Y_pred, axis=1)
count = 0
for i in range(len(Y_pred)):
    if Y_pred[i] == Y_test[i]:
        count += 1
print(count)
accuracy = count/np.shape(Y_test)[0]
print('Test Accuracy: ',accuracy)

Y_pred = CNN.predict(X_valid,batch_size=300)
Y_pred = np.argmax(Y_pred, axis=1)
count = 0
for i in range(len(Y_pred)):
    if Y_pred[i] == Y_valid[i]:
        count += 1
print(count)
accuracy = count/np.shape(Y_valid)[0]
print('Validation Accuracy: ',accuracy)

kernel_list:  [5, 5] 
stride_list:  [1, 1] 
learn_rate:  0.001
Epoch 1/5
14400/14400 [==============================] - 4s 267us/step - loss: 0.6366 - acc: 0.6667
Epoch 2/5
14400/14400 [==============================] - 3s 242us/step - loss: 0.6347 - acc: 0.6667
Epoch 3/5
14400/14400 [==============================] - 3s 239us/step - loss: 0.6332 - acc: 0.6667
Epoch 4/5
14400/14400 [==============================] - 3s 239us/step - loss: 0.6320 - acc: 0.6667
Epoch 5/5
14400/14400 [==============================] - 4s 244us/step - loss: 0.6309 - acc: 0.6667
1900
Test Accuracy:  0.6333333333333333
1514
Validation Accuracy:  0.42055555555555557


FMNIST6

In [23]:
labels = np.unique(FMNIST6['Str'])
num_label = len(labels)

X_test = torch.Tensor(FMNIST6['Xts']).unsqueeze(-1)/256 # Normalization
X_test.type(torch.float64)
Y_test_in = FMNIST6['Yts']

X_input = torch.Tensor(FMNIST6['Xtr']).unsqueeze(-1)/256 # Normalization
X_input.type(torch.float64)
Y_input = FMNIST6['Str']

num_img = X_input.size(0)
idx_list = torch.randperm(num_img)
sep_point = int(num_img*0.8)
X_train = X_input[idx_list[:sep_point],:,:,:].permute(0,3,1,2)
Y_train_in = Y_input[idx_list[:sep_point]]
X_valid = X_input[idx_list[sep_point:],:,:,:].permute(0,3,1,2)
Y_valid_in = Y_input[idx_list[sep_point:]]
X_test = X_test.permute(0,3,1,2)

Y_test_exp = np.zeros([Y_test_in.shape[0],num_label],dtype = float)
Y_train_exp = np.zeros([Y_train_in.shape[0],num_label],dtype = float)

for i in range(Y_train_exp.shape[0]):

    label_train_tmp = Y_train_in[i]
    Y_train_exp[i,label_train_tmp] = 1

for i in range(Y_test_exp.shape[0]):

    label_train_tmp = Y_test_in[i]
    Y_test_exp[i,label_train_tmp] = 1




X_train = X_train.permute(0,2,3,1).cpu().numpy()
X_test = X_test.permute(0,2,3,1).cpu().numpy()
X_valid = X_valid.permute(0,2,3,1).cpu().numpy()
Y_test = np.array(Y_test_in)
Y_valid = np.array(Y_valid_in)

CNN = create_CNN()

CNN.fit(X_train,Y_train_exp,batch_size=300,epochs=5)

Y_pred = CNN.predict(X_test,batch_size=300)
Y_pred = np.argmax(Y_pred, axis=1)
count = 0
for i in range(len(Y_pred)):
    if Y_pred[i] == Y_test[i]:
        count += 1
print(count)
accuracy = count/np.shape(Y_test)[0]
print('Test Accuracy: ',accuracy)

Y_pred = CNN.predict(X_valid,batch_size=300)
Y_pred = np.argmax(Y_pred, axis=1)
count = 0
for i in range(len(Y_pred)):
    if Y_pred[i] == Y_valid[i]:
        count += 1
print(count)
accuracy = count/np.shape(Y_valid)[0]
print('Validation Accuracy: ',accuracy)

kernel_list:  [5, 5] 
stride_list:  [1, 1] 
learn_rate:  0.001
Epoch 1/5
14400/14400 [==============================] - 4s 261us/step - loss: 0.6400 - acc: 0.6667
Epoch 2/5
14400/14400 [==============================] - 3s 239us/step - loss: 0.6376 - acc: 0.6667
Epoch 3/5
14400/14400 [==============================] - 3s 240us/step - loss: 0.6366 - acc: 0.6667
Epoch 4/5
14400/14400 [==============================] - 3s 242us/step - loss: 0.6360 - acc: 0.6667
Epoch 5/5
14400/14400 [==============================] - 4s 248us/step - loss: 0.6357 - acc: 0.6667
1870
Test Accuracy:  0.6233333333333333
1358
Validation Accuracy:  0.37722222222222224
